- title: Immutability Principles
- author: Oscar Vargas Torres
- date: 2018-08-27
- category: Functional Programming
- tags: Immutability

## Mutability is hard to reason about

A lot of programming languages support mutability. For example, Python can mutate objects in place (as well as JavaScript and a lot others):

In [1]:
x = [1, 2, 3]

In [2]:
x.reverse()
x

[3, 2, 1]

This may not seem problematic at first. A lot of people would argue that it is indeed necessary to program. However, when things can change, we sometimes are forced to understand more details than the bare minimum necessary. For example:

In [3]:
# This function is just for ilustration purposes.
# Imagine a situation where a very long and complex method mutates one of it's arguments...
def m1(x):
    """Reverses its argument"""
    return x.reverse()

vowels = ['a', 'e', 'i', 'o', 'u'] 
m1(vowels)
vowels

['u', 'o', 'i', 'e', 'a']

Now, we have to dig into the implementation of `m1`, to understand how the method affects its arguments.

A simpler approach is to rely on immutable data structures/variables. This may seem like a more difficult approach, but it makes programming easier in the long run.

Let's use an immutable approach to the previous problem with `pyrsistent` Python's library:

In [4]:
from pyrsistent import plist
ns1 = plist([1, 2, 3])
ns1

plist([1, 2, 3])

In [5]:
ns2 = ns1.reverse()
ns2

plist([3, 2, 1])

In [6]:
# Notice that original list remains unmodified (it is a immutable/persistent data structure!)
ns1

plist([1, 2, 3])

## Functional programming relies on immutability

Immutable data structures/collections exist in a lot of programming languages:
* Haskell
    * https://haskell-containers.readthedocs.io/en/latest/intro.html
* Scala
    * https://docs.scala-lang.org/overviews/collections/overview.html    
* FSharp
    * https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/fsharp-collection-types
* Clojure
    * https://clojure.org/reference/data_structures
* C#
    * https://msdn.microsoft.com/en-us/library/system.collections.immutable(v=vs.111).aspx
    * https://msdn.microsoft.com/en-us/magazine/mt795189.aspx
* JavaScript
    * https://facebook.github.io/immutable-js/
* etc